### Code to save svgs with a defined size and font

In [ ]:
# For pheatmap
pheatmap(heatmap.combo, filename = paste(fig_dir, 'cell_class_heatmap.pdf', sep='/'), width = 3.4, height = 1.6, fontsize = 7)

In [ ]:
# For ggsave
ggsave(gautier_UMAP, filename = paste(fig_dir, 'gautier_UMAP.svg', sep='/'), device='svg', width = 3, height = 3, units = "in")

In [ ]:
# For pheatmap, width = 3.2 in, height 4 in or less
# For scatter plots, width = height = 1 in
# Font = 7

In [1]:
renv::init('/oak/stanford/groups/agitler/Shared/Shared_Jupyter_Notebook_Analysis/4.1.1-OG/')

Warning message:
“The following package(s) are missing entries in the cache:

	remotes

These packages will need to be reinstalled.
”


In [2]:
library(dplyr)
library(Seurat)
library(ggplot2)
library(ggrepel)
#library(ComplexHeatmap)
library(pheatmap)
library(gplots)
library(svglite)
library(hdf5r)
library(DropletUtils)
library(scDblFinder)
library(scuttle)
library(scales)
library(ggrastr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRange

In [3]:
library(future)
plan("multicore", workers = availableCores())
options(future.globals.maxSize= +Inf)
# NotebookApp.max_buffer_size=214748364800


Attaching package: ‘future’


The following object is masked from ‘package:SummarizedExperiment’:

    values


The following object is masked from ‘package:GenomicRanges’:

    values


The following object is masked from ‘package:S4Vectors’:

    values


The following object is masked from ‘package:hdf5r’:

    values




### File Loading and Saving Directories

In [4]:
load.dir <- '/oak/stanford/groups/agitler/Shared_Sequencing/Jacob/Levine-Human-FastQ/h5_files'
fig_dir <- '/oak/stanford/groups/agitler/Shared/Matters_Arising/Figure_Panels'
csv_dir <- '/oak/stanford/groups/agitler/Shared/Matters_Arising/CSV_Files'

### Load H5 Matrices from Yadav 2022

In [5]:
donor1_h5 <- Read10X_h5(paste(load.dir,'GSM5723843_Donor1_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)
donor2_h5 <- Read10X_h5(paste(load.dir,'GSM5723844_Donor2_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)
donor3_h5 <- Read10X_h5(paste(load.dir,'GSM5723845_Donor3_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)
donor4_h5 <- Read10X_h5(paste(load.dir,'GSM5723846_Donor4_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)
donor5_h5 <- Read10X_h5(paste(load.dir,'GSM5723847_Donor5_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)
donor6_h5 <- Read10X_h5(paste(load.dir,'GSM5723848_Donor6_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)
donor7_h5 <- Read10X_h5(paste(load.dir,'GSM5723849_Donor7_raw_feature_bc_matrix.h5',sep='/'), use.names = TRUE, unique.features = TRUE)


Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”
Warning message in spa

In [6]:
bcrank1 <- barcodeRanks(donor1_h5)
bcrank2 <- barcodeRanks(donor2_h5)
bcrank3 <- barcodeRanks(donor3_h5)
bcrank4 <- barcodeRanks(donor4_h5)
bcrank5 <- barcodeRanks(donor5_h5)
bcrank6 <- barcodeRanks(donor6_h5)
bcrank7 <- barcodeRanks(donor7_h5)

In [7]:
debris_donor1 <- donor1_h5[,donor1_h5['NEFH', ] >1 & bcrank1$total<2000 & bcrank1$total>100]
donor1.debris.seurat <- CreateSeuratObject(debris_donor1)

In [8]:
debris_donor2 <- donor2_h5[,donor2_h5['NEFH', ] >1 & bcrank2$total<2000 & bcrank2$total>100]
donor2.debris.seurat <- CreateSeuratObject(debris_donor2)

In [9]:
debris_donor3 <- donor3_h5[,donor3_h5['NEFH', ] >1 & bcrank3$total<2000 & bcrank3$total>100]
donor3.debris.seurat <- CreateSeuratObject(debris_donor3)

In [10]:
debris_donor4 <- donor4_h5[,donor4_h5['NEFH', ] >1 & bcrank4$total<2000 & bcrank4$total>100]
donor4.debris.seurat <- CreateSeuratObject(debris_donor4)

In [11]:
debris_donor5 <- donor5_h5[,donor5_h5['NEFH', ] >1 & bcrank5$total<2000 & bcrank5$total>100]
donor5.debris.seurat <- CreateSeuratObject(debris_donor5)

In [12]:
debris_donor6 <- donor6_h5[,donor6_h5['NEFH', ] >1 & bcrank6$total<2000 & bcrank6$total>100]
donor6.debris.seurat <- CreateSeuratObject(debris_donor6)

In [13]:
debris_donor7 <- donor7_h5[,donor7_h5['NEFH', ] >1 & bcrank7$total<2000 & bcrank7$total>100]
donor7.debris.seurat <- CreateSeuratObject(debris_donor7)

### Log Normalize list of Seurat Objects

In [14]:
source('/oak/stanford/groups/agitler/Jacob/single_cell_motor_neurons/seurat_analysis_scripts/sc_functions.R')

In [15]:
debris.objects.list <- c(donor1.debris.seurat, donor2.debris.seurat, donor3.debris.seurat, donor4.debris.seurat, donor5.debris.seurat, donor6.debris.seurat, donor7.debris.seurat)
debris.objects.list.normalized <- normalize.aggregated.object(debris.objects.list)

[1] "splitting complete"
[1] 1
[1] "normalization complete"


Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -3.0007”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.30103”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  3.025e-15”


[1] "variable feature selection complete"
[1] 2
[1] "normalization complete"


Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -2.7132”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.32138”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  7.9239e-29”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“There are other near singularities as well. 0.090619”


[1] "variable feature selection complete"
[1] 3
[1] "normalization complete"


...

[1] "variable feature selection complete"
[1] 4
[1] "normalization complete"


Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -2.6375”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.32149”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  3.9454e-29”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“There are other near singularities as well. 0.090619”


[1] "variable feature selection complete"
[1] 5
[1] "normalization complete"


Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -2.0897”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.31887”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  2.6703e-28”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“There are other near singularities as well. 0.031008”


[1] "variable feature selection complete"
[1] 6
[1] "normalization complete"
[1] "variable feature selection complete"
[1] 7
[1] "normalization complete"
[1] "variable feature selection complete"


### Remove Mitochondrial Reads

In [16]:
for(i in 1:length(debris.objects.list.normalized)) {
    counts <- GetAssayData(debris.objects.list.normalized[[i]], assay = "RNA")
    mito_genes <- grep( "^MT-", rownames(debris.objects.list.normalized[[i]]), value = T)
    counts <- counts[-(which(rownames(counts) %in% mito_genes)),]
    debris.objects.list.normalized[[i]] <- subset(debris.objects.list.normalized[[i]], features = rownames(counts))
    
}

### Find Variable Features etc

In [17]:
for(i in 1:length(debris.objects.list.normalized)) {
    DefaultAssay(debris.objects.list.normalized[[i]]) <- 'RNA'
    debris.objects.list.normalized[[i]] <- FindVariableFeatures(debris.objects.list.normalized[[i]])
    debris.objects.list.normalized[[i]] <- find.pcs.and.clustids(debris.objects.list.normalized[[i]])
    
}

Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -3.0007”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.30103”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  1.2356e-14”
Loading required package: foreach

Loading required package: iterators

Loading required package: parallel

Centering and scaling data matrix

PC_ 1 
Positive:  ARHGAP24, KIAA1328, PLXDC2, LINC01608, LINC00862, RNF219-AS1, CLMN, C3, GRID2, CDK19 
	   PCDH9-AS2, MIR2052HG, RORB, ZNF131, GRIK1, APBB1IP, HIF1A-AS2, NEK11, ZHX2, FAM66D 
	   BCL2L14, AOAH, ARHGAP15, MEF2C, PDE7A, RGCC, ELMO1-AS1, LINC01184, FOXO1, ZNF862 
Negative:  NEFM, NEFL, MAP1B, HSP90AA1, TUBB2A, UCHL1, HSP90AB1, STMN2, NEFH, PRPH 
	   KIF21A, YWHAG, PKM, ATP1B1, VAT1L, RTN4, THY1, RTN3, ACTG1, AKAP12 
	   TUBB4B, DYNC1H1, RTN1, SV2C, CLU, TUBA1B, SNC

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2003
Number of edges: 142348

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7667
Number of communities: 3
Elapsed time: 0 seconds


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -2.7132”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.32138”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  4.3742e-28”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“There are other near singulariti

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 493
Number of edges: 35524

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.6242
Number of communities: 2
Elapsed time: 0 seconds


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“pseudoinverse used at -2.3959”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“neighborhood radius 0.32036”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“reciprocal condition number  6.7313e-28”
Warning message in simpleLoess(y, x, w, span, degree = degree, parametric = parametric, :
“There are other near singulariti

...

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Centering and scaling data matrix

PC_ 1 
Positive:  NEFM, NEFL, NEFH, HSP90AA1, MAP1B, UCHL1, HSP90AB1, PKM, THY1, TUBB2A 
	   STMN2, CLU, PRPH, KLC1, ATP1A1, ATP1B1, ENO2, EEF1A2, YWHAG, RTN1 
	   RTN3, SV2C, TUBA1B, SNAP25, ANXA6, NDRG4, VAT1L, CADM3, GAPDH, TUBB4B 
Negative:  NRG3, MIR99AHG, PLEK, C10orf90, DOCK4, SGCZ, FKBP5, SLC1A3, ETV6, ARHGAP26 
	   KCNQ3, FAM193A, KIAA0586, KIRREL3, ENOX1, LRMDA, VAV1, CEMIP, MIR646HG, MTRNR2L8 
	   RNF220, INPP5D, LDLRAD4, RUNX1, PEAK1, DHRS3, IL17

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 1841
Number of edges: 118140

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7368
Number of communities: 4
Elapsed time: 0 seconds


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”
Centering and scaling data matrix

PC_ 1 
Positive:  MALAT1, NEFH, ST18, SLC44A1, FRMD4B, LRMDA, IL1RAPL1, GFAP, GAB1, DOCK10 
	   FCER1G, S100A11, ZEB2, ARL15, TCF12, FTX, ZBTB20, SAMSN1, ITPR2, VRK2 
	   KCNH8, AC092691.1, PDE1A, TBXAS1, EFEMP1, CD69, FMNL2, UNC5C, CNDP1, SRGN 
Negative:  ATP1B1, INA, HSP90AA1, MAP1B, NSF, VSNL1, STXBP1, NEFM, DNM1, ANK2 
	   SYT2, CLSTN1, GABRG2, SLC6A5, SCN1A, LAMP5, ATP2B2, SLC22A17, NEFL, SV2A 
	   PEG10, CKB, HAPLN4, OXR1, CAMTA1, SLC32A1, IDS, SCG2, C

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 3433
Number of edges: 174123

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7920
Number of communities: 6
Elapsed time: 0 seconds


1 singletons identified. 5 final clusters.

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”


### Integrate Across Datasets

In [18]:
debris.droplets.anchors <- FindIntegrationAnchors(object.list = debris.objects.list.normalized, )


Warning message in CheckDuplicateCellNames(object.list = object.list):
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”
Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 2441 anchors

Filtering anchors

	Retained 1576 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 1189 anchors

Filtering anchors

	Retained 1115 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 1155 anchors

Filtering anchors

	Retained 1089 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 2061 anchors

Filtering anchors

	Retained 1425 anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 1744 anchors

Filtering anchors

	Retained 1200 anchors

Running CCA

Merging objects

Finding neighborhoods

Fin

In [19]:
debris.droplets.integrated <- IntegrateData(anchorset = debris.droplets.anchors, verbose=T)


Merging dataset 5 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 3 into 1 5

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 4 into 1 5 3

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 2 into 1 5 3 4

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 6 into 1 5 3 4 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 7 into 1 5 3 4 2 6

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data



In [20]:
DefaultAssay(debris.droplets.integrated) <- 'RNA'
debris.droplets.integrated <- FindVariableFeatures(debris.droplets.integrated)
debris.droplets.integrated <- find.pcs.and.clustids(debris.droplets.integrated)

Centering and scaling data matrix

PC_ 1 
Positive:  NEAT1, DLG2, LRP1B, ERBB4, NRG3, NCAM2, DLC1, DPP10, DOCK4, MIR99AHG 
	   RNF220, MAML2, GPC5, SGCZ, CNTNAP2, LRMDA, KLHL32, ANKS1B, LRRC4C, FAM155A 
	   KIRREL3, RBFOX1, FRMD4A, SAMD12, ENOX1, NRXN1, ARHGAP26, GALNT13, LAMA2, TTC28 
Negative:  NEFL, NEFM, HSP90AB1, HSP90AA1, UCHL1, NEFH, TUBB2A, STMN2, MAP1B, HSPA8 
	   PKM, ATP1B1, CALM1, S100A10, DYNC1H1, GNAS, PRPH, MAP7D2, DNAJA1, VAT1L 
	   AKAP12, ANXA2, HSPB1, AHNAK2, PVALB, UTS2, MAP9, NSF, PLD3, DNM1 
PC_ 2 
Positive:  MAP1B, ATP1B1, PKM, NEFM, NSF, HSP90AA1, DYNC1H1, SYT2, HSP90AB1, DNM3 
	   PLD3, CLSTN1, MAP7D2, DNM1, NEFL, DPP6, SCN1A, TUBB2A, STXBP1, UCHL1 
	   GABRG2, AKAP12, RBFOX1, STMN2, NRXN1, VAT1L, NCDN, GNAS, CALM1, CADPS 
Negative:  NEFH, AC105402.3, GFAP, HBB, S100A11, DHFR, MTRNR2L12, HIF1A-AS2, AC024230.1, HBA2 
	   VAMP8, AL358216.1, AC024257.1, AC015923.1, DHDH, AL353768.1, AC022613.1, SULT1C4, LPAR4, AC138956.2 
	   GPC2, MPZ, AC007114.1, SLC13A5, AC0036

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 8540
Number of edges: 357615

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8605
Number of communities: 9
Elapsed time: 0 seconds


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”


### Based on expression of NEAT1 and NEFM, Clusters 2, 4, and 5 are neuritic debris

In [21]:
DefaultAssay(debris.droplets.integrated) <- 'integrated'
debris.droplets.integrated.final <- subset(debris.droplets.integrated, idents=c(2,3,5))

In [22]:
DefaultAssay(debris.droplets.integrated) <- 'RNA'
debris.droplets.integrated.final <- FindVariableFeatures(debris.droplets.integrated.final)
debris.droplets.integrated.final <- find.pcs.and.clustids(debris.droplets.integrated.final)

Warning message in FindVariableFeatures.Assay(object = assay.data, selection.method = selection.method, :
“selection.method set to 'vst' but count slot is empty; will use data slot instead”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in hvf.info$variance.expected[not.const] <- 10^fit$fitted:
“number of items to replace is not a multiple of replacement length”
Centering and scaling data matrix

PC_ 1 
Positive:  MALAT1, LINC02082, ZNF665, AC079465.1, AC080162.1, E2F5, AC008074.1, AL031280.1, PRRX1, AL133480.1 
	   ANO2, LINC01482, AC009690.1, C10orf90, AC007495.2, KIRREL3, TRG-AS1, IL12A-AS1, AC104596.1, NFKB1 
	   RAD54L2, SOX5, ECM2, RHBDF2, NAMPT, KAT6A, AC008758.4, SYCE2, BMPR1B, DIAPH3 
Negative:  NEFM, NEFL, MAP1B, HSP90AA1, NEFH, HSP90AB1, UCHL1, RTN4, GAPDH, RTN1 
	   STMN2, YWHAG, RTN3, CLU, TUBB2A, SNAP25, DYNC1H1, SV2C, PRNP, AKAP12 
	   KIF21A, ACTG1, ENO2, MAP7D2, EPB41L3, PKM, TUBB4B, PRPH, VAT1L, ACLY 
PC_ 2 
Positive:  PRPH, TRG-AS1, S10

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2908
Number of edges: 185073

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8236
Number of communities: 7
Elapsed time: 0 seconds


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”


In [23]:
DefaultAssay(debris.droplets.integrated.final) <- 'integrated'
debris.droplets.integrated.neurites <- subset(debris.droplets.integrated.final, idents=c(0,1,2))

In [24]:
debris.droplets.integrated.neurites <- FindVariableFeatures(debris.droplets.integrated.neurites)
debris.droplets.integrated.neurites <- RunPCA(debris.droplets.integrated.neurites)


Warning message in FindVariableFeatures.Assay(object = assay.data, selection.method = selection.method, :
“selection.method set to 'vst' but count slot is empty; will use data slot instead”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in hvf.info$variance.expected[not.const] <- 10^fit$fitted:
“number of items to replace is not a multiple of replacement length”
PC_ 1 
Positive:  NEFL, NEFM, NEFH, MAP1B, PRPH, UCHL1, ACTG1, HSP90AA1, VAT1L, MCAM 
	   ANXA2, GAPDH, TUBB2A, HSP90AB1, STMN2, CLU, MAP7D2, RTN1, S100A10, RTN4 
	   LGALS1, ENO2, KLC1, SLC5A7, KIF21A, ACLY, KIF3A, AKAP12, EPB41L3, DYNC1H1 
Negative:  MALAT1, E2F5, LINC02082, AC008074.1, AC079465.1, LINC01482, AL133480.1, DIAPH3, IL12A-AS1, ZNF331 
	   GALNTL6, RGS22, C10orf90, RHBDF2, KAT6A, AC009690.1, TOX3, AL031280.1, VSNL1, NLGN1 
	   AC104596.1, ZNF665, RNF220, AL139294.1, TC2N, ANO2, SOX5, COBLL1, AC009975.1, SUFU 
PC_ 2 
Positive:  INA, SLC6A5, LAMP5, VSNL1, AC104041.1, SLC32A1, NSF, ATP1

In [25]:
debris.droplets.integrated.neurites <- FindNeighbors(debris.droplets.integrated.neurites, dims = 1:20)
debris.droplets.integrated.neurites <- FindClusters(debris.droplets.integrated.neurites, resolution = 2)

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2005
Number of edges: 105217

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.5901
Number of communities: 10
Elapsed time: 0 seconds


Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”


In [26]:
debris.droplets.integrated.neurites <- RunUMAP(debris.droplets.integrated.neurites, dims = 1:20)

16:39:06 UMAP embedding parameters a = 0.9922 b = 1.112

16:39:06 Read 2005 rows and found 20 numeric columns

16:39:06 Using Annoy for neighbor search, n_neighbors = 30

16:39:06 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

16:39:06 Writing NN index file to temp file /tmp/RtmpMBkzmG/file646350d59651

16:39:06 Searching Annoy index using 8 threads, search_k = 3000

16:39:07 Annoy recall = 100%

16:39:08 Commencing smooth kNN distance calibration using 8 threads
 with target n_neighbors = 30

16:39:12 Initializing from normalized Laplacian + noise (using irlba)

16:39:12 Commencing optimization for 500 epochs, with 82886 positive edges

16:39:19 Optimization finished



In [27]:
neurite_debris_umap <- DimPlot(debris.droplets.integrated.neurites, label=T, raster = F, pt.size = 1.5)  + NoLegend()
neurite_debris_umap <- rasterize(neurite_debris_umap, layers='Point', dpi=500)
ggsave(filename = paste(fig_dir,'neurite_debris_umap.svg', sep='/'),
       plot= neurite_debris_umap, device = 'svg')

Saving 6.67 x 6.67 in image



In [28]:
DefaultAssay(debris.droplets.integrated.neurites) <- 'RNA'

neurite_debris_featureplot <- FeaturePlot(debris.droplets.integrated.neurites, pt.size = 1,
                                          features=c('SLC17A6', 'SLC6A5', 'SLC5A7', 'GAD1'), label=T) 
neurite_debris_featureplot <- rasterize(neurite_debris_featureplot, layers='Point', dpi=500)

ggsave(filename = paste(fig_dir,'neurite_debris_featureplot.svg', sep='/'),
       plot= neurite_debris_featureplot, device = 'svg')

Saving 6.67 x 6.67 in image



In [37]:
neurite.exp.mat <- AverageExpression(debris.droplets.integrated.neurites, assays = 'RNA')$RNA

In [38]:
marker.heatmap <- neurite.exp.mat[c('SLC5A7', 'SLC17A6', 
                                    'SLC6A5', 'GAD1', 'ACLY','GAD2'),]

In [39]:
marker.heatmap <- (marker.heatmap - rowMins(marker.heatmap))/(rowMaxs(marker.heatmap) - rowMins(marker.heatmap))

In [40]:
colnames(marker.heatmap) <- paste('Cluster', colnames(marker.heatmap), sep=' ')

In [31]:
# anno <- data.frame('cluster'= paste('cluster', colnames(marker.heatmap)))

In [32]:
# colpal <- hue_pal()(10)
# names(colpal) <- anno[,1]

In [47]:
neurite_subclustering_heatmap <- 
    pheatmap(marker.heatmap, cluster_rows = TRUE, cluster_cols = TRUE, 
             filename = paste(fig_dir, 'neurite_subclustering_heatmap.pdf', sep='/'), 
             width = 6.4, height = 5.5, fontsize = 14) 

In [48]:
# svglite(filename = paste(fig_dir, 'neurite_subclustering.svg', sep='/'))
# pheatmap(marker.heatmap,  scale='row', annotation_col = anno, show_colnames = FALSE, annotation_colors = list(cluster=colpal))

# dev.off()

In [106]:
cholinergic_glutamatergic_scatter <- FeatureScatter(debris.droplets.integrated.neurites, pt.size=0.2, feature1=c('SLC5A7'), feature2='SLC17A6')
cholinergic_glutamatergic_scatter <- rasterize(cholinergic_glutamatergic_scatter, layer= 'Point', dpi=500) + 
    NoLegend() + FontSize(x.title=7, y.title=7, x.text=7, y.text=7) + theme(plot.title=element_blank())

In [107]:
cholinergic_glycinergic_scatter <- FeatureScatter(debris.droplets.integrated.neurites, pt.size = 0.2, feature1=c('SLC5A7'), feature2='SLC6A5')
cholinergic_glycinergic_scatter <- rasterize(cholinergic_glycinergic_scatter, layer= 'Point', dpi=500) + 
    NoLegend() + FontSize(x.title=7, y.title=7, x.text=7, y.text=7) + theme(plot.title=element_blank())

In [108]:
cholinergic_gabaergic_scatter <- FeatureScatter(debris.droplets.integrated.neurites, pt.size=0.2, feature1=c('SLC5A7'), feature2='GAD1')
cholinergic_gabaergic_scatter <- rasterize(cholinergic_gabaergic_scatter, layer= 'Point', dpi=500) +
    NoLegend() + FontSize(x.title=7, y.title=7, x.text=7, y.text=7) + theme(plot.title=element_blank())

In [109]:
ggsave(filename = paste(fig_dir,'cholinergic_glutamatergic_scatter.svg', sep='/'),
       plot= cholinergic_glutamatergic_scatter, device = 'svg', width=1, height=1, unit= 'in')

In [110]:
ggsave(filename = paste(fig_dir,'cholinergic_glycinergic_scatter.svg', sep='/'),
       plot= cholinergic_glycinergic_scatter, device = 'svg', width=1, height=1, unit= 'in')

In [111]:
ggsave(filename = paste(fig_dir,'cholinergic_gabaergic_scatter.svg', sep='/'),
       plot= cholinergic_gabaergic_scatter, device = 'svg', width=1, height=1, unit= 'in')

In [37]:
saveRDS(debris.droplets.integrated.neurites, file= '/oak/stanford/groups/agitler/Shared/Matters_Arising/Gautier_Yadav/debris_droplets_integrated_neurites.RDS')


In [5]:
debris.droplets.integrated.neurites <- readRDS(file= '/oak/stanford/groups/agitler/Shared/Matters_Arising/Gautier_Yadav/debris_droplets_integrated_neurites.RDS')
